In [1]:
!pip install fuzzywuzzy

In [41]:
import fitz  # PyMuPDF
import os
from fuzzywuzzy import fuzz

def fuzzy_pdf_highlighter(pdf_path, search_string):
    """
    Highlights a sentence based on a fuzzy search across the entire PDF document.

    Parameters:
    pdf_path (str): The path to the PDF document.
    search_string (str): The string to search for in the document.

    Returns:
    str: The path to the modified PDF document.
    """

    doc = fitz.open(pdf_path)
    output_pdf_path = "highlighted_documents"
    if not os.path.exists(output_pdf_path):
        os.makedirs(output_pdf_path)

    filename = os.path.basename(pdf_path)
    filepath = os.path.join(output_pdf_path, filename)


    full_text = ""
    for page in doc:
        full_text += page.get_text() + " "
    words = full_text.split()

    search_length = len(search_string.split())

    # rolling window
    chunks = [' '.join(words[i:i+search_length]) for i in range(len(words)-search_length+1)]
    match_score, match_chunk = max((fuzz.ratio(search_string, chunk), chunk) for chunk in chunks)
    
    print("Match score:", match_score)
    if match_score > 85:  # Adjust score_cutoff as needed
        print("Match found:", match_chunk)

        for page in doc:
            text_instances = page.search_for(match_chunk)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)

    doc.save(filepath, garbage=4, deflate=True)
    doc.close()
    return filepath

# Example usage
pdf_path = "text1.PDF"
search_string = """4ej  länder  skog78kötsefff    ln  å  allmänningsskogen  till  men  eller 

om
eljest synnerliga skäl föreligga till avyttringen."""

search_string = """Bestämmelserna i 13 och        14 §§ skall dock fortfarande tillämpas i fråga


om mark som införlivats med-4      allmänningsskog genom förordnande som har




meddelats före lagens ikraftträdande."""


fuzzy_pdf_highlighter(pdf_path, search_string)

Match score: 94
Match found: Bestämmelserna i 13 och 14 §§ skall dock fortfarande tillämpas i fråga om mark som införlivats med allmänningsskog genom förordnande som har meddelats före lagens ikraftträdande.


'highlighted_documents/text1.PDF'